In [ ]:
#Imports
import pandas as pd
from pathlib import Path
import os
import numpy as np


In [ ]:
TRACK_DIR = Path("train")
WEEK_FMT  = "oriented_2023_w{week:02d}.csv"

In [ ]:
#Read in Data
projectRoot = Path.cwd()
dfPlays = pd.read_csv(f'{projectRoot}/features/selectedPlays.csv')

In [ ]:
to_num = lambda s: pd.to_numeric(s, errors="coerce")


In [ ]:
def load_week(week: int) -> pd.DataFrame:
    f = TRACK_DIR / WEEK_FMT.format(week=week)
    return pd.read_csv(f) if f.exists() else pd.DataFrame()

In [ ]:
def get_throw_frame_last(trk_play: pd.DataFrame) -> int | None:
    # Throw is the last frame in your tracking
    return int(trk_play["frameId"].max()) if len(trk_play) else None

In [ ]:
def mask_qb(df: pd.DataFrame) -> pd.Series:
    # prefer role 'Passer', fallback to position 'QB'
    role = df["player_role"].astype(str).str.lower()
    pos  = df["player_position"].astype(str).str.upper()
    return role.eq("passer") | pos.eq("QB")

In [ ]:
def mask_targeted(df: pd.DataFrame) -> pd.Series:
    # prefer role 'Targeted Receiver', fallback to player_to_predict == True
    role = df["player_role"].astype(str).str.lower()
    if "player_to_predict" in df.columns:
        return role.eq("targeted receiver") | df["player_to_predict"].fillna(False).astype(bool)
    return role.eq("targeted receiver")

In [ ]:
def mask_defense(df: pd.DataFrame) -> pd.Series:
    return df["player_side"].astype(str).str.lower().eq("defense")

In [ ]:
def dist_xy_row(a: pd.Series, b: pd.Series) -> float:
    return float(np.hypot(a["x"] - b["x"], a["y"] - b["y"]))

In [ ]:
def nearest_defender_sep(frame0: pd.DataFrame, wr_row: pd.Series) -> float:
    defs = frame0[mask_defense(frame0)]
    if defs.empty: return np.nan
    dists = defs.apply(lambda r: dist_xy_row(wr_row, r), axis=1)
    return float(dists.min())

In [ ]:
def density_within(frame0: pd.DataFrame, wr_row: pd.Series, radius: float) -> int:
    defs = frame0[mask_defense(frame0)]
    if defs.empty: return 0
    return int((defs.apply(lambda r: dist_xy_row(wr_row, r), axis=1) <= radius).sum())

In [ ]:
def nearest_defender(frame0: pd.DataFrame, wr_row: pd.Series):
    defs = frame0[mask_defense(frame0)]
    if defs.empty: 
        return np.nan, None
    dists = defs.apply(lambda r: dist_xy_row(wr_row, r), axis=1)
    j = dists.idxmin()
    return float(dists.loc[j]), defs.loc[j]

In [ ]:
def _vel_components(row: pd.Series):
    if "s" not in row or "dir" not in row:
        return np.nan, np.nan
    s   = float(pd.to_numeric(row["s"], errors="coerce"))
    ang = float(pd.to_numeric(row["dir"], errors="coerce"))
    if not np.isfinite(s) or not np.isfinite(ang):
        return np.nan, np.nan
    rad = np.deg2rad(ang)
    return s*np.cos(rad), s*np.sin(rad)

In [ ]:
def closing_speed_defender(wr_row: pd.Series, def_row: pd.Series) -> float:
    # unit vector from DEF -> WR
    dx = float(wr_row["x"] - def_row["x"])
    dy = float(wr_row["y"] - def_row["y"])
    dist = float(np.hypot(dx, dy))
    if dist == 0:
        return np.nan
    ux, uy = dx/dist, dy/dist

    vwx, vwy = _vel_components(wr_row)
    vdx, vdy = _vel_components(def_row)
    if not (np.isfinite(vwx) and np.isfinite(vwy) and np.isfinite(vdx) and np.isfinite(vdy)):
        return np.nan

    # dr/dt = (v_wr - v_def) · u; closing speed = -(dr/dt) = (v_def - v_wr) · u
    return (vdx - vwx)*ux + (vdy - vwy)*uy

In [ ]:
def features_at_throw_lastframe(frame0: pd.DataFrame, play_row: pd.Series) -> dict | None:
    qb = frame0[mask_qb(frame0)].head(1)
    receiver = frame0[mask_targeted(frame0)].head(1)
    if receiver.empty:
        off_mask = frame0["player_side"].astype(str).str.lower().eq("offense")
        receiver = frame0[off_mask & (~mask_qb(frame0))].head(1)
    if qb.empty or receiver.empty:
        return None

    qb, receiver = qb.iloc[0], receiver.iloc[0]

    # nearest defender + speed + closing
    sep0, def_row = nearest_defender(frame0, receiver)
    def1_speed   = float(pd.to_numeric(def_row["s"], errors="coerce")) if def_row is not None and "s" in frame0.columns else np.nan
    def1_closing = closing_speed_defender(receiver, def_row) if def_row is not None else np.nan

    # Receiver speed
    receiver_speed = float(pd.to_numeric(frame0.loc[receiver.name, "s"], errors="coerce")) if "s" in frame0.columns else np.nan

    # target (ball landing) & derived geometry
    tx = float(frame0["ball_land_x"].iloc[0]) if "ball_land_x" in frame0.columns else np.nan
    ty = float(frame0["ball_land_y"].iloc[0]) if "ball_land_y" in frame0.columns else np.nan
    target_depth = float(np.hypot(tx - qb["x"], ty - qb["y"])) if np.isfinite(tx) and np.isfinite(ty) else np.nan
    receiver_to_target = float(np.hypot(tx - receiver["x"], ty - receiver["y"])) if np.isfinite(tx) and np.isfinite(ty) else np.nan
    receiver_to_def1 = float(np.hypot(receiver["x"] - def_row["x"], receiver["y"] - def_row["y"])) if def_row is not None and np.isfinite(def_row["x"]) and np.isfinite(def_row["y"]) else np.nan


    return {
        "game_id":                int(play_row["game_id"]),
        "play_id":                int(play_row["play_id"]),
        "week":                   int(play_row["week"]),
        # at-throw features
        "receiver_sep0_yd":       sep0,
        "def1_speed0_mps":        def1_speed,
        "def1_closing_mps":       def1_closing,     # + means defender is closing
        "receiver_speed0_mps":    receiver_speed,
        "def_density_r15":        density_within(frame0, receiver, 1.5),
        "def_density_r30":        density_within(frame0, receiver, 3.0),
        # target geometry
        "ball_land_x":            tx,
        "ball_land_y":            ty,
        "target_depth_yd":        target_depth,
        "receiver_to_target_yd":  receiver_to_target,
        # placeholder
    }


In [ ]:
# dfPlays must have: week, game_id, play_id
plays_sorted = dfPlays.sort_values(["week","game_id","play_id"]).reset_index(drop=True)

all_feats = []
for week in plays_sorted["week"].dropna().astype(int).unique():
    trk_w = load_week(week)
    if trk_w.empty:
        continue

    wk_plays = plays_sorted[plays_sorted["week"].astype(int).eq(week)]
    for _, prow in wk_plays.iterrows():
        gid, pid = int(prow["game_id"]), int(prow["play_id"])
        trk_play = trk_w[(trk_w["game_id"]==gid) & (trk_w["play_id"]==pid)]
        if trk_play.empty:
            continue

        f_throw = get_throw_frame_last(trk_play)
        if f_throw is None:
            continue

        frame0 = trk_play[trk_play["frame_id"]==f_throw]
        if frame0.empty:
            continue

        feat = features_at_throw_lastframe(frame0, prow)
        if feat:
            all_feats.append(feat)

features_df = pd.DataFrame(all_feats)
